In [1]:
import numpy as np
import collections

### 问题
对一个数组 nums 和一个整数 k ，返回其中出现频率前 k 高的元素

### Demo Case

In [2]:
nums = [1,1,1,1,1,2,2,3,3,3,3,3,3,3,6,6,6,6,4,4,4,4,4,4,9,7,7,7,7]
k = 3

- 统计不重复元素的频数  $O(n)$

In [3]:
stat = collections.Counter(nums)
stat = list(stat.items()) #频次列表
stat

[(1, 5), (2, 2), (3, 7), (6, 4), (4, 6), (9, 1), (7, 4)]

### 0.baseline 基于排序（快排）

In [4]:
def BaseSort(nums,k,stat):
    #stat = collections.Counter(nums)
    #stat = list(stat.items())    
    stat_sorted = sorted(stat,key = lambda x:x[1],reverse=True)
    return stat_sorted[:k]
    #print(stat_sorted)

In [5]:
BaseSort(nums,k,stat)

[(3, 7), (4, 6), (1, 5)]

### 1.基于堆（小顶堆）
时间复杂度：$O(nlogk)$
空间复杂度：$O(k)$

In [6]:
#取父/子节点基本操作
def lchild(node):
    return node << 1
def rchild(node):
    return node << 1 | 1
def father(node):
    return node >> 1
#上浮节点，用于向堆插入新节点
def heap_up(heap,node):
    val = heap[node]
    while father(node)>0 and val[1]<heap[father(node)][1]:
        heap[node] = heap[father(node)]
        node = father(node)
    heap[node] = val
#下沉节点，用于调整堆
def heap_down(heap,node,k):
    root = node #root作为变量逐步下沉
    val = heap[node] #存储node原值
    while lchild(root) <= k:
        child = lchild(root) #先选取左子节点
        if rchild(root) <= k and child < rchild(root): #如果右更小，选取右子节点
            child = rchild(root)
        #验证确实当前节点值大于选取的子节点，则交换
        if heap[child][1] < val[1]:
            heap[root] = heap[child]
            root = child
        else: #否则则找到位置，结束循环
            break
    heap[root] = val #最后赋值
#堆排序输出
def heap_sort(heap):
    for i in range(len(heap)-1, 0, -1):
        heap[1], heap[i] = heap[i], heap[1]
        heap_down(heap, 1, i)
#主函数
def HeapFrequent(nums,k,stat):
    #stat = collections.Counter(nums)
    #stat = list(stat.items())
    heap = [(0,0)] #占位上界
    #使用heap_up()建堆 规模：k+1(包括占位上界) 
    for i in range(k):
        heap.append(stat[i])
        heap_up(heap, len(heap)-1)
    #使用heap_down()维护堆 新元素大于堆顶即下沉
    for i in range(k,len(stat)):
        if stat[i][1] > heap[1][1]: #heap[1]为堆顶
            heap[1] = stat[i] #去除原根（堆顶）节点
            heap_down(heap,1,k) #将node下沉
    #使用heap_sort()排序，并倒序得到从大到小结果
    heap_sort(heap)
    result = heap[1:][::-1]
    return result

In [7]:
HeapFrequent(nums,k,stat)

[(3, 7), (4, 6), (1, 5)]

### 2.基于桶排序
哈希空间集中时，时间、空间复杂度均接近$O(n)$ 
分散时则增添很多无谓循环开销

In [8]:
def BucketFrequent(nums,k,spacesize,stat):
    #stat = collections.Counter(nums)
    #stat = list(stat.items())
    Bucket = [[] for i in range(spacesize)]
    for s in stat:
        Bucket[s[1]].append(s)
        
    r = 0
    flag = False
    result = []
    for i in range(len(Bucket)-1,-1,-1):
        if not flag and Bucket[i] != []:
            for s in Bucket[i]:
                result.append(s)
                r += 1
                if r == k:
                    flag = True
                    break
    return result

In [9]:
spacesize = 10
BucketFrequent(nums,k,spacesize,stat)

[(3, 7), (4, 6), (1, 5)]

### 大量数据实验

In [11]:
import random
import time
import math

client_num = 10000000
hash_space = int(client_num / 500)

hash_list = [random.randint(1, hash_space) for i in range(client_num)]
hash_k = 3

st = time.perf_counter()
stat = collections.Counter(hash_list)
stat = list(stat.items()) #频次列表
et = time.perf_counter()
print("Count costs: {:.4f}s".format((et-st)))

st = time.perf_counter()
base_result = BaseSort(hash_list,hash_k,stat)
print("Base Sort:")
print(base_result)
et = time.perf_counter()
print("Base Sort costs: {:.4f}s".format((et-st)))


st = time.perf_counter()
heap_result = HeapFrequent(hash_list,hash_k,stat)
print("Heap:")
print(heap_result)
et = time.perf_counter()
print("HeapFrequent costs: {:.4f}s".format((et-st)))

st = time.perf_counter()
bucket_result = BucketFrequent(hash_list,hash_k,hash_space,stat)
print("Bucket:")
print(bucket_result)
et = time.perf_counter()
print("BucketFrequent costs: {:.4f}s".format((et-st)))

Count costs: 0.5779s
Base Sort:
[(16829, 598), (3143, 587), (10715, 586)]
Base Sort costs: 0.0029s
Heap:
[(16829, 598), (3143, 587), (1971, 518)]
HeapFrequent costs: 0.0015s
Bucket:
[(16829, 598), (3143, 587), (10715, 586)]
BucketFrequent costs: 0.0035s


- 统计

In [15]:
s_power = list(range(6,12))
s_k = list(range(1,10))
base_datas = np.zeros((len(s_power),len(s_k)))
count_datas = np.zeros((len(s_power),len(s_k)))
heap_datas = np.zeros((len(s_power),len(s_k)))
bucket_datas = np.zeros((len(s_power),len(s_k)))

In [26]:

for p in range(3,4):
    client_num = 10 ** s_power[p]
    hash_space = int(client_num / 500)
    hash_list = [random.randint(1, hash_space) for i in range(client_num)]
    for j in range(len(s_k)):
        if count_datas[p][j] == 0:
            hash_k = s_k[j]
            st = time.perf_counter()
            stat = collections.Counter(hash_list)
            stat = list(stat.items()) #频次列表
            #print(stat)
            et = time.perf_counter()
            count_datas[p][j] = et-st
            print("Count costs: {:.4f}s".format((et-st)))
            print(j,end = ',')
            
            st = time.perf_counter()
            base_result = BaseSort(hash_list,hash_k,stat)
            et = time.perf_counter()
            base_datas[p][j] = et-st
            
            st = time.perf_counter()
            heap_result = HeapFrequent(hash_list,hash_k,stat)
            et = time.perf_counter()
            heap_datas[p][j] = et-st

            st = time.perf_counter()
            bucket_result = BucketFrequent(hash_list,hash_k,hash_space,stat)
            et = time.perf_counter()
            bucket_datas[p][j] = et-st

            print(k,end = ',')

MemoryError: 

In [53]:
math.log(client_num)

16.11809565095832

In [25]:
list(range(3))


[0, 1, 2]

In [18]:
print(count_datas)

[[ 0.0350243  0.0350931  0.0355179  0.0353041  0.0354417  0.0344587
   0.0344811  0.0351545  0.0346498]
 [ 0.5843058  0.52981    0.5306835  0.9529362  0.5071953  0.5195359
   0.5112306  0.533784   0.9476811]
 [17.0368663 15.9008414 15.8126735 16.0458307 16.3124926 16.4702285
  16.3545783 16.2321129 17.4694494]
 [ 0.         0.         0.         0.         0.         0.
   0.         0.         0.       ]
 [ 0.         0.         0.         0.         0.         0.
   0.         0.         0.       ]
 [ 0.         0.         0.         0.         0.         0.
   0.         0.         0.       ]]


In [23]:
for i in range(3):
    for j in range(9):
        #print(count_datas[i][j]," ",heap_datas[i][j]," ",bucket_datas[i][j],end=" ")
        print(base_datas[i][j],heap_datas[i][j],bucket_datas[i][j])
    print()

0.0002661000000898639 0.00015159999998104468 0.004588699999999335
0.00025560000005953043 0.00016019999998206913 0.00036330000000361906
0.00025909999999385036 0.00016180000000076689 0.00039079999999103165
0.0002607000000125481 0.00016279999999824213 0.37855210000009265
0.00026890000003731984 0.00016360000006443443 0.00036069999998744606
0.0002575999999407941 0.000176199999941673 0.0003575000000637374
0.0002555999999458436 0.00018569999997453124 0.0003507999999783351
0.00025699999991957156 0.00018499999998766725 0.0003553000000238171
0.0002527000000327462 0.0001907000000755943 0.0003513999999995576

0.00276239999993777 0.001393699999994169 0.4374801999999818
0.0027131999999028267 0.0014453999999659572 0.0033086000000821514
0.003304899999989175 0.002739500000075168 0.0054337999999916065
0.0028180000000475047 0.0014470999999502965 0.0033238999999412044
0.002597000000037042 0.001356700000087585 0.003269100000011349
0.002919000000019878 0.0015452000000095722 0.44065309999996316
0.00285020000